In [ ]:
!pip install arxiv fpdf requests

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=97a09906acf60473f4c4677abf66c4c611532a7240cc6df9837fd99d3619d37c
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=0279225a66f73c869117ef38bb7d64500223e15f2d51f3f37420bdd28f05577c
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built fpdf sgmllib3k


In [ ]:
OPENROUTER_API_KEY = "ENTER-YOUR-OPENROUTER-API-KEY"
OPENROUTER_MODEL = "qwen/qwen3-coder:free"

In [ ]:
import arxiv

def search_arxiv_papers(query, max_results=5):
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    papers = []
    for result in search.results():
        papers.append({
            "title": result.title,
            "summary": result.summary,
            "pdf_url": result.pdf_url
        })
    return papers

In [ ]:
import requests

def summarize_with_openrouter(text):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": OPENROUTER_MODEL,
        "messages": [
            {"role": "system", "content": "You are a helpful academic research assistant. Summarize academic abstracts clearly and concisely."},
            {"role": "user", "content": f"Summarize the following abstract:\n\n{text}"}
        ],
        "temperature": 0.7,
        "max_tokens": 1024
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"❌ Error {response.status_code}: {response.text}"

In [ ]:
def generate_literature_review(topic, papers):
    report = f"# Literature Review on: {topic}\n\n"
    for i, paper in enumerate(papers):
        title = paper['title']
        print(f"🔄 Summarizing paper {i+1}: {title}")
        summary = summarize_with_openrouter(paper['summary'])
        link = paper['pdf_url']
        report += f"### {i+1}. {title}\n\n"
        report += f"**Summary:** {summary}\n\n"
        report += f"**PDF Link:** [View Paper]({link})\n\n---\n\n"
    report += "*Generated by Autonomous Research Assistant using OpenRouter AI.*"
    return report

In [ ]:
from fpdf import FPDF

def save_report_as_pdf(report_text, filename="literature_review.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in report_text.split('\n'):
        pdf.multi_cell(0, 10, line)
    pdf.output(filename)
    print(f"✅ PDF saved: {filename}")

In [ ]:
research_topic = "Multimodal AI"
papers = search_arxiv_papers(research_topic, max_results=5)
report_text = generate_literature_review(research_topic, papers)
save_report_as_pdf(report_text)

/tmp/ipython-input-3-4176154379.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


🔄 Summarizing paper 1: Toward AI-driven Multimodal Interfaces for Industrial CAD Modeling
🔄 Summarizing paper 2: Navigating the landscape of multimodal AI in medicine: a scoping review on technical challenges and clinical applications
🔄 Summarizing paper 3: Multimodal foundation models are better simulators of the human brain
🔄 Summarizing paper 4: Multimodal Conversational AI: A Survey of Datasets and Approaches
🔄 Summarizing paper 5: Towards a Multimodal Document-grounded Conversational AI System for Education
✅ PDF saved: literature_review.pdf
